<a href="https://colab.research.google.com/github/yura-eng/HUFSworkshop/blob/main/gradio_app_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Sequence rearrangement

In [ ]:
!pip install ipywidgets

import random
import gradio as gr

# Define the sentences
sentences = [
    "In the village of Lila, there was an ancient 'Tree of Dreams' believed to make whispered dreams come true.",
    "A young artist, Maria, used whatever materials she could find to create her art due to her family's poverty.",
    "Encouraged by her teacher to enter an art competition, Maria whispered her wish to the Tree of Dreams.",
    "The next day, Maria received donated art supplies, enabling her to create a beautiful, winning painting.",
    "Maria's victory inspired the village to believe in their dreams, renewing faith in the legendary tree's magic."
]

# Shuffle the sentences
shuffled_sentences = random.sample(sentences, len(sentences))

def display_sentences():
    display_text = "Here are the shuffled sentences:\n\n"
    for i, sentence in enumerate(shuffled_sentences):
        display_text += f"{chr(65 + i)}: {sentence}\n\n"
    return display_text

def check_order(user_input):
    # Convert user input to uppercase
    user_input = user_input.upper()

    # Convert the user input to a list of indices
    user_order = [ord(char) - 65 for char in user_input]

    # Generate the correct order based on the original sentences
    correct_order_indices = [shuffled_sentences.index(sentence) for sentence in sentences]

    # Check if the user's order matches the correct order
    is_correct = user_order == correct_order_indices

    # Prepare the feedback
    if is_correct:
        return "Correct! You have identified the correct order of the sentences."
    else:
        return "Incorrect. Try again."

# Create Gradio interface
def interface():
    sentences_display = display_sentences()
    return gr.Interface(fn=check_order, inputs=gr.Textbox(label="Enter the correct order (e.g., 'ABDEC')", placeholder="Type the order here..."), outputs="text", live=True).launch(share=True)

app = gr.Blocks()

with app:
    gr.Markdown(display_sentences())
    with gr.Row():
        order_input = gr.Textbox(label="Enter the correct order (e.g., 'ABDEC')", placeholder="Type the order here...")
        output = gr.Textbox(label="Feedback")
    order_input.change(fn=check_order, inputs=order_input, outputs=output)

app.launch()

##Synomym quiz

In [ ]:
!pip install gradio

import gradio as gr
import random

# Define questions and answers
questions = ["colorful", "worried", "excited", "showing", "respected", "doubters", "success"]
answers = ["vibrant", "anxious", "thrilled", "depicting", "admired", "skeptics", "achievement"]

answered_questions = set()  # Track which questions have been correctly answered

def get_options(question):
    index = questions.index(question)
    correct_answer = answers[index]
    options = set([correct_answer])
    while len(options) < 4:
        options.add(random.choice(answers))
    options = list(options)
    random.shuffle(options)
    return ", ".join(options), correct_answer, index

def check_answer(user_input, correct_answer, question_index):
    if question_index in answered_questions:
        return "Correct! You have already answered this question correctly.", 0
    if user_input.strip().lower() == correct_answer.lower():
        feedback = "Correct!"
        score = 1
        answered_questions.add(question_index)  # Only add to answered if correct
    else:
        feedback = f"Incorrect. The correct answer was '{correct_answer}'."
        score = 0
    return feedback, score

with gr.Blocks() as app:
    with gr.Row():
        question_dropdown = gr.Dropdown(choices=questions, label="Select a question")
        show_options_button = gr.Button("Show me options")
    options_output = gr.Textbox(label="Options", interactive=False)
    correct_answer_store = gr.State()
    question_index_store = gr.State()
    user_input = gr.Textbox(label="Type your answer here", visible=True)
    submit_button = gr.Button("Submit Answer", visible=True)
    feedback_label = gr.Label()
    score_store = gr.State(value=0)
    complete_button = gr.Button("Complete and Show Total Score", visible=True)

    def show_options(question):
        options, correct, index = get_options(question)
        options_output.value = options
        correct_answer_store.value = correct
        question_index_store.value = index
        return options, correct, index

    show_options_button.click(
        fn=show_options,
        inputs=[question_dropdown],
        outputs=[options_output, correct_answer_store, question_index_store]
    )

    submit_button.click(
        fn=lambda user_input, correct_answer_store=correct_answer_store, question_index_store=question_index_store, score_store=score_store: submit_response(user_input, correct_answer_store.value, question_index_store.value, score_store),
        inputs=user_input,
        outputs=[feedback_label, score_store]
    )

    def submit_response(user_input, correct_answer, question_index, score_store):
        feedback, score = check_answer(user_input, correct_answer, question_index)
        if score == 1:
            score_store.value += score  # Only accumulate the score if it's a new correct answer
        return feedback, score_store.value

    def complete_and_reset(score_store):
        final_score = score_store.value
        score_store.value = 0  # Reset the score
        answered_questions.clear()  # Clear the set of answered questions
        return f"Session Complete. Your final score is {final_score}."

    complete_button.click(
        fn=lambda score_store=score_store: complete_and_reset(score_store),
        inputs=[],
        outputs=feedback_label
    )

app.launch()


##Gradio Cloze Question App

In [ ]:
!pip install gradio gtts pydub

import gradio as gr
import requests
from gtts import gTTS
from pydub import AudioSegment
from pydub.generators import Sine
import os

# Cloze questions
cloze_questions = [
    {
        "question": "Alex [BLANK] for his daring spirit and his love for exploring the rugged landscapes around Echo Ridge.",
        "answer": "was known",
        "hint": "know"
    },
    {
        "question": "One day, while exploring the local library, Alex [BLANK] upon an ancient map tucked inside a forgotten book on village lore.",
        "answer": "stumbled",
        "hint": "stumble"
    },
    {
        "question": "The map [BLANK] at the location of a lost treasure, hidden deep within a cave known as Whispering Hollow.",
        "answer": "hinted",
        "hint": "hint"
    },
    {
        "question": "Excited by the prospect of a real adventure, Alex [BLANK] to seek out the treasure.",
        "answer": "decided",
        "hint": "decide"
    },
    {
        "question": "Knowing the journey would be risky, he [BLANK] the help of his best friends, Mia and Sam.",
        "answer": "enlisted",
        "hint": "enlist"
    },
    {
        "question": "Together, they [BLANK] for the expedition, gathering supplies and studying the map extensively.",
        "answer": "prepared",
        "hint": "prepare"
    },
    {
        "question": "They [BLANK] their route, took note of landmarks, and readied themselves for any challenges they might face.",
        "answer": "planned",
        "hint": "plan"
    }
]

# Function to create a bell sound
def create_bell_sound(filename="bell.wav"):
    # Generate a bell sound (1000 Hz tone for 200 ms)
    tone = Sine(1000).to_audio_segment(duration=200).apply_gain(-10).fade_in(50).fade_out(50)
    tone.export(filename, format="wav")

# Create the bell sound file
create_bell_sound()

# Function to convert text to speech and add bell sound
def text_to_speech_with_bell(text, filename):
    tts = gTTS(text.replace("[BLANK]", ""))
    tts.save("temp.mp3")

    # Load the generated speech and bell sound
    speech = AudioSegment.from_mp3("temp.mp3")
    bell = AudioSegment.from_wav("bell.wav")

    # Find the position of the blank and insert the bell sound
    blank_position = text.find("[BLANK]")
    part1 = text[:blank_position]
    part2 = text[blank_position + len("[BLANK]"):]
    tts_part1 = gTTS(part1).save("part1.mp3")
    tts_part2 = gTTS(part2).save("part2.mp3")
    speech_part1 = AudioSegment.from_mp3("part1.mp3")
    speech_part2 = AudioSegment.from_mp3("part2.mp3")

    # Create a silent segment (2 seconds)
    silent_segment = AudioSegment.silent(duration=2000)

    combined = speech_part1 + bell + silent_segment + speech_part2

    # Save the final audio with the bell sound inserted
    combined.export(filename, format="mp3")
    os.remove("temp.mp3")
    os.remove("part1.mp3")
    os.remove("part2.mp3")
    return filename

# Function to convert text to speech without any modifications
def text_to_speech(text, filename):
    tts = gTTS(text)
    tts.save(filename)
    return filename

# Generate audio files for questions
for i, question in enumerate(cloze_questions):
    full_audio_filename = f"full_question_{i+1}.mp3"
    cloze_audio_filename = f"cloze_question_{i+1}.mp3"
    question["full_audio"] = text_to_speech(question["question"].replace("[BLANK]", question["answer"]), full_audio_filename)
    question["cloze_audio"] = text_to_speech_with_bell(question["question"], cloze_audio_filename)

# Function to handle the cloze quiz
def cloze_quiz(state, answer):
    name, question_index, score, results = state
    question = cloze_questions[question_index]

    correct = answer.strip().lower() == question["answer"].lower()

    if correct:
        score += 1
        results.append(f"Question {question_index + 1}: Correct\n")
    else:
        results.append(f"Question {question_index + 1}: Incorrect, the correct answer is: {question['answer']}\n")

    question_index += 1

    if question_index < len(cloze_questions):
        next_full_audio = cloze_questions[question_index]["full_audio"]
        next_cloze_audio = cloze_questions[question_index]["cloze_audio"]
        next_hint = f"Hint: {cloze_questions[question_index]['hint']} ◁◁◁ Check out this verb!"
        return (name, question_index, score, results), next_full_audio, next_cloze_audio, next_hint, gr.update(visible=False), gr.update(value="", interactive=True, visible=True)
    else:
        result_text = f"* Name: {name}\n* Score: {score} out of {len(cloze_questions)}\n" + "\n".join(results)
        return (name, question_index, score, results), None, None, "", gr.update(visible=True, value=result_text), gr.update(visible=False)

# Function to start the quiz
def start_quiz(name):
    hint = f"Hint: {cloze_questions[0]['hint']} ◁◁◁ Check out this verb!"
    return (name, 0, 0, []), cloze_questions[0]["full_audio"], cloze_questions[0]["cloze_audio"], hint, gr.update(visible=False), gr.update(visible=True)

# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# Listening Cloze Test Instructions")
    gr.Markdown("""
    **Instructions:**

    1. Listen to the full sentence.
    2. Listen to the cloze question with the beep sound.
    3. Write the correct form of the verb you hear in the blank space. Remember, you do not need to write the entire sentence, just the verb.

    **For example, if you hear "Yesterday, Alex went to the store," and then you hear "Yesterday, Alex ___ to the store" with a beep sound, you should write "went" if that is the verb you heard.**

    **Are you ready? Let's begin!**
    """)

    name_input = gr.Textbox(label="Enter your name")
    start_button = gr.Button("Start Quiz")
    full_question_audio = gr.Audio(interactive=True, autoplay=False, label="Full Sentence Listening")
    cloze_question_audio = gr.Audio(interactive=True, autoplay=False, label="Cloze Question")
    hint_output = gr.Markdown()
    answer_input = gr.Textbox(label="Your Answer")
    next_button = gr.Button("Next")
    result_output = gr.Textbox(label="Results", interactive=False, visible=False)

    # Initialize the state
    state = gr.State()

    start_button.click(start_quiz, inputs=name_input, outputs=[state, full_question_audio, cloze_question_audio, hint_output, result_output, answer_input])
    next_button.click(cloze_quiz, inputs=[state, answer_input], outputs=[state, full_question_audio, cloze_question_audio, hint_output, result_output, answer_input])

iface.launch(share=True)


##Gradio Writing Checker App

In [ ]:
# app.py
!pip install transformers gradio nltk sentencepiece torch

import shutil
import nltk
from transformers import T5Tokenizer, T5ForConditionalGeneration
import gradio as gr
from difflib import SequenceMatcher

# Remove and re-download punkt resources to ensure no corrupted files
nltk.data.path.append("/usr/local/share/nltk_data/")
shutil.rmtree('/usr/local/share/nltk_data/tokenizers/punkt', ignore_errors=True)
nltk.download("punkt", download_dir="/usr/local/share/nltk_data/")

# Load a pre-trained T5 model specifically fine-tuned for grammar correction
try:
    tokenizer = T5Tokenizer.from_pretrained("prithivida/grammar_error_correcter_v1")
    model = T5ForConditionalGeneration.from_pretrained("prithivida/grammar_error_correcter_v1")
except Exception as e:
    print(f"Error loading model: {e}")

# Function to perform grammar correction and highlight changes
def grammar_check(text):
    try:
        # Split the text by periods for a basic sentence split
        sentences = text.split(". ")
        corrected_sentences = []

        # Process each sentence individually
        for sentence in sentences:
            input_text = f"gec: {sentence}"
            input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
            outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
            corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Compare the original and corrected sentences and highlight changes
            highlighted_sentence = highlight_changes(sentence, corrected_sentence)
            corrected_sentences.append(highlighted_sentence)

        # Join corrected sentences into a single string with line breaks
        corrected_text = "\n".join(corrected_sentences)
        return corrected_text  # Return plain text with highlighted changes

    except Exception as e:
        # Return the error message as output to help diagnose
        return f"An error occurred during processing: {e}"

# Function to highlight differences between original and corrected sentences
def highlight_changes(original, corrected):
    diff = SequenceMatcher(None, original.split(), corrected.split())
    result = []
    for tag, i1, i2, j1, j2 in diff.get_opcodes():
        if tag == 'replace' or tag == 'insert':
            result.append(f"**{' '.join(corrected.split()[j1:j2])}**")  # Mark changed words in bold
        elif tag == 'equal':
            result.append(' '.join(original.split()[i1:i2]))  # Keep unchanged words
    return " ".join(result)

# Create Gradio interface with a writing prompt
interface = gr.Interface(
    fn=grammar_check,
    inputs="text",
    outputs="text",  # Use text output for reliable display in Colab
    title="Grammar Checker",
    description=(
        "Enter text to check for grammar mistakes.\n\n"
        "Writing Prompt:\n"
        "In the story, Alex and his friends discovered an ancient treasure in Whispering Hollow and decided to donate the artifacts to the local museum.\n\n"
        "In the past, did you have a similar experience where you found something valuable or interesting? Tell the story. Describe what you found, what you did with it, and how you felt about your decision.\n\n"
        "Remember to use past tense in your writing.\n\n"
        "A student's sample answer:\n"
        "When I was 10, I find an old coin in my backyard. I kept it for a while and shows it to my friends. They was impressed and say it might be valuable. Later, I take it to a local antique shop, and the owner told me it was very old. I decided to give it to the museum in my town. The museum was happy and put it on display. I feel proud of my decision."
    )
)

# Launch the interface
interface.launch()


In [ ]:
!pip install gradio

import gradio as gr

def quiz(question1, question2, question3, question4):
    score = 0
    total_questions = 3  # Adjust if you change the number of scored questions

    responses = {
        'Question 1: What is the name of the main character?': question1,
        'Question 2: What is the name of the village?': question2,
        'Question 3: What did Maria have a passion for?': question3,
        'Question 4: Your dreams can come true with belief, hard work, and a little bit of what?': question4
    }

    correct_answers = {
        'Question 1: What is the name of the main character?': 'maria',
        'Question 2: What is the name of the village?': 'lila',
        'Question 3: What did Maria have a passion for?': 'art',
        'Question 4: Your dreams can come true with belief, hard work, and a little bit of what?': 'luck'  # Adjust this if needed
    }

    output = []
    for question, user_answer in responses.items():
        if user_answer.lower() == correct_answers[question]:
            score += 1
            output.append(f"{question} - Correct!")
        else:
            output.append(f"{question} - Wrong Answer :(")

    mark = (score / total_questions) * 100
    output.append(f"Thank you for taking this small quiz, you answered {score} questions correctly!")
    output.append(f"Marks obtained: {mark}%")
    output.append("BYE!")
    return "\n".join(output)

iface = gr.Interface(
    fn=quiz,
    inputs=[gr.Textbox(label="Question 1: What is the name of the main character?"),
            gr.Textbox(label="Question 2: What is the name of the village?"),
            gr.Textbox(label="Question 3: What did Maria have a passion for?"),
            gr.Textbox(label="Question 4: Your dreams can come true with belief, hard work, and a little bit of what?")],
    outputs="text",
    title="G1 Listening Quiz",
    description="Welcome to G1 Listening Quiz - Remember to only use lowercase letters. Are you ready to take the quiz?"
)

iface.launch()